<h1><center>EN.553.688: Computing for Applied Mathematics - Final Assignment</center></h1>
<h1><center>Dayou Ren</center></h1>

# Steps and ideas

Read in train data frame.

## Data frame sanity check

Check the dimensions of the data frame.

Check for the existence of NaNs in the data frame.

## Preprocessing

Add edited_2023 column.

Imputation: subsitute NaNs in each column with the median in that column.

Scale the data.

## Split data frame

Split the data frame into 1:4. 20% testing and 80% training.

## Regression and Classification Models:

### For length prediction
Linear Regression

Decision Tree Regression

### For word_present prediction:

Decision Tree (with threshold adjustment)

Logistic Regression (with threshold adjustment)

Support Vector Machine with Linear Kernel and class weights adjustment

K-th Nearest Neighbor

Random Forest with threshold adjustment

### For edited_2023 prediction:

Decision Tree (with threshold adjustment)

Logistic Regression (with threshold adjustment)

Support Vector Machine with Linear Kernel and class weights adjustment

K-th Nearest Neighbot

Random Forest with threshold adjustment

## Model Selection

Decision Tree Regression is a better option for length predictions compared to Linear Regression.

It seems like Random Forest is the best option for word_present and edited_2023 predictions with the constraint of lowering FPR.

## Apply trained models to test dataset
Read in test data frame. Same preprocessing necessary as above.

Apply models.

Export to a CSV file.

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix, mean_absolute_error, precision_recall_curve, roc_curve
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier

In [2]:
df_train = pd.read_pickle('./training_data.pkl')
# dataset sanity check
print(f'Shape: {df_train.shape}')
nan_counts = df_train.isna().sum()
print(nan_counts)
df_train.head()

Shape: (200000, 44)
URLID              0
educat          4031
polit           3977
people          4000
anger           4083
human           4022
excess          3944
city            4044
ocean           4044
math            3917
social          3948
rest            4042
gram            4031
analysis        4102
data            3961
process         4001
problem         4003
example         4087
sales           4020
product         4051
question        3917
qualit          3879
categor         4161
first           3905
transaction     4029
survey          4037
optim           3921
number          4018
conduct         4000
experienc       3904
visual          4062
audio           4082
elect           3959
decision        3991
petrol          4089
mouse           4108
danger          3963
heart           3885
soul            4032
grade           4079
elderly         3975
length             0
date               0
word_present       0
dtype: int64


,URLID,educat,polit,people,anger,human,excess,city,ocean,math,...,petrol,mouse,danger,heart,soul,grade,elderly,length,date,word_present
243838,URLID_HLGM,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15118,2023-03-07,0
180391,URLID_TIQI,2.0,0.0,0.0,2.0,22.0,3.0,16.0,0.0,2.0,...,0.0,0.0,0.0,2.0,0.0,1.0,0.0,142859,2023-03-26,1
107367,URLID_EVZN,0.0,3.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,46591,2023-01-01,1
19660,URLID_SBDC,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,40552,2023-02-02,1
48203,URLID_NHNM,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,35071,2023-02-03,0


In [3]:
# imputation
selected_columns = df_train.columns[1:-3]
medians = df_train[selected_columns].median()
df_train[selected_columns] = df_train[selected_columns].fillna(medians)
filled_nan_counts = df_train.isna().sum()
print(filled_nan_counts)

# add edited_2023
df_train['date'] = pd.to_datetime(df_train['date'])
df_train['edited_2023'] = (df_train['date'].dt.year == 2023).astype(int)

# check training sample distributions
print(df_train['word_present'].value_counts())
print(df_train['edited_2023'].value_counts())
df_train.head()

URLID           0
educat          0
polit           0
people          0
anger           0
human           0
excess          0
city            0
ocean           0
math            0
social          0
rest            0
gram            0
analysis        0
data            0
process         0
problem         0
example         0
sales           0
product         0
question        0
qualit          0
categor         0
first           0
transaction     0
survey          0
optim           0
number          0
conduct         0
experienc       0
visual          0
audio           0
elect           0
decision        0
petrol          0
mouse           0
danger          0
heart           0
soul            0
grade           0
elderly         0
length          0
date            0
word_present    0
dtype: int64
word_present
1    127531
0     72469
Name: count, dtype: int64
edited_2023
1    137628
0     62372
Name: count, dtype: int64


,URLID,educat,polit,people,anger,human,excess,city,ocean,math,...,mouse,danger,heart,soul,grade,elderly,length,date,word_present,edited_2023
243838,URLID_HLGM,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,15118,2023-03-07,0,1
180391,URLID_TIQI,2.0,0.0,0.0,2.0,22.0,3.0,16.0,0.0,2.0,...,0.0,0.0,2.0,0.0,1.0,0.0,142859,2023-03-26,1,1
107367,URLID_EVZN,0.0,3.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,46591,2023-01-01,1,1
19660,URLID_SBDC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,40552,2023-02-02,1,1
48203,URLID_NHNM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,35071,2023-02-03,0,1


In [4]:
# split the dataframe
feature = df_train.iloc[:, 1:-4]
scaler = StandardScaler()
feature = scaler.fit_transform(feature)
response = df_train.iloc[:, -4:]

print(df_train.columns.tolist())

f_train, f_test, r_train, r_test = train_test_split(feature, response, test_size=0.2)
print(f_train.shape)
print(f_test.shape)

r_train.head()

['URLID', 'educat', 'polit', 'people', 'anger', 'human', 'excess', 'city', 'ocean', 'math', 'social', 'rest', 'gram', 'analysis', 'data', 'process', 'problem', 'example', 'sales', 'product', 'question', 'qualit', 'categor', 'first', 'transaction', 'survey', 'optim', 'number', 'conduct', 'experienc', 'visual', 'audio', 'elect', 'decision', 'petrol', 'mouse', 'danger', 'heart', 'soul', 'grade', 'elderly', 'length', 'date', 'word_present', 'edited_2023']
(160000, 40)
(40000, 40)


,length,date,word_present,edited_2023
144062,19331,2021-01-09,1,0
172958,47949,2023-03-23,1,1
144499,24984,2023-02-19,1,1
125899,36188,2023-01-12,0,1
70240,23018,2023-03-11,0,1


In [5]:
# Isolate 'length' response variable
length_r_train = r_train['length']
length_r_test = r_test['length']

In [6]:
# Linear Regression for length prediction
model_lr = LinearRegression()
model_lr.fit(f_train, length_r_train)

r_pred = model_lr.predict(f_test)
mae = mean_absolute_error(length_r_test, r_pred)

print(f'MAE: {mae}')

MAE: 8754.398345611608


In [7]:
# Decision Tree Regression for length prediction
model_dtr = DecisionTreeRegressor()
model_dtr.fit(f_train, length_r_train)

r_pred = model_dtr.predict(f_test)
mae = mean_absolute_error(length_r_test, r_pred)

print(f'MAE: {mae}')

MAE: 7598.373634157763


In [8]:
# Isolate 'word_present' response variable
wp_r_train = r_train['word_present']
wp_r_test = r_test['word_present']

In [9]:
# Decision Tree
dtc_wp = DecisionTreeClassifier()
dtc_wp.fit(f_train, wp_r_train)

r_pred = dtc_wp.predict(f_test)

conf_matrix = confusion_matrix(wp_r_test, r_pred)/f_test.shape[0]

print(f'Confusion Matrix:\n{conf_matrix}')

print(f'True Positive Rate: {conf_matrix[1][1]/(conf_matrix[1][1]+conf_matrix[1][0])}')

print(f'False Positive Rate: {conf_matrix[0][1]/(conf_matrix[0][0]+conf_matrix[0][1])}')

print()

# adjust threshold
probabilities = dtc_wp.predict_proba(f_test)[:, 1]

fpr, tpr, threshold = roc_curve(wp_r_test, probabilities)

selected_threshold = threshold[np.argmin(np.abs(fpr - 0.05))]

r_pred_adjusted = (probabilities >= selected_threshold).astype(int)

conf_matrix_adjusted = confusion_matrix(wp_r_test, r_pred_adjusted)/f_test.shape[0]

print(f'Adjusted Confusion Matrix:\n{conf_matrix_adjusted}')
print(f'True Positive Rate: {conf_matrix_adjusted[1][1]/(conf_matrix_adjusted[1][1]+conf_matrix_adjusted[1][0])}')
print(f'False Positive Rate: {conf_matrix_adjusted[0][1]/(conf_matrix_adjusted[0][0]+conf_matrix_adjusted[0][1])}')

Confusion Matrix:
[[0.239875 0.12045 ]
 [0.156325 0.48335 ]]
True Positive Rate: 0.755618087309962
False Positive Rate: 0.33428155137722887

Adjusted Confusion Matrix:
[[0.360325 0.      ]
 [0.639675 0.      ]]
True Positive Rate: 0.0
False Positive Rate: 0.0


In [10]:
# Logistic Regression
model_logr_wp = LogisticRegression(max_iter=1000, class_weight='balanced')
model_logr_wp.fit(f_train, wp_r_train)

r_pred = model_logr_wp.predict(f_test)

conf_matrix = confusion_matrix(wp_r_test, r_pred)/f_test.shape[0]

print(f'Confusion Matrix:\n{conf_matrix}')

print(f'True Positive Rate: {conf_matrix[1][1]/(conf_matrix[1][1]+conf_matrix[1][0])}')

print(f'False Positive Rate: {conf_matrix[0][1]/(conf_matrix[0][0]+conf_matrix[0][1])}')

print()

# adjust threshold
probabilities = model_logr_wp.predict_proba(f_test)[:, 1]

fpr, tpr, threshold = roc_curve(wp_r_test, probabilities)

selected_threshold = threshold[np.argmin(np.abs(fpr - 0.05))]

r_pred_adjusted = (probabilities >= selected_threshold).astype(int)

conf_matrix_adjusted = confusion_matrix(wp_r_test, r_pred_adjusted)/f_test.shape[0]

print(f'Adjusted Confusion Matrix:\n{conf_matrix_adjusted}')
print(f'True Positive Rate: {conf_matrix_adjusted[1][1]/(conf_matrix_adjusted[1][1]+conf_matrix_adjusted[1][0])}')
print(f'False Positive Rate: {conf_matrix_adjusted[0][1]/(conf_matrix_adjusted[0][0]+conf_matrix_adjusted[0][1])}')

Confusion Matrix:
[[0.289825 0.0705  ]
 [0.267325 0.37235 ]]
True Positive Rate: 0.5820924688318287
False Positive Rate: 0.19565669881357106

Adjusted Confusion Matrix:
[[0.3423   0.018025]
 [0.44365  0.196025]]
True Positive Rate: 0.3064446789385235
False Positive Rate: 0.05002428363283147


In [11]:
# Support Vector Machine
svm_wp = LinearSVC(max_iter=100000, dual=False, class_weight={0:1, 1:0.32})
svm_wp.fit(f_train, wp_r_train)

r_pred = svm_wp.predict(f_test)

conf_matrix = confusion_matrix(wp_r_test, r_pred)/f_test.shape[0]

print(f'Confusion Matrix:\n{conf_matrix}')
print(f'True Positive Rate: {conf_matrix[1][1]/(conf_matrix[1][1]+conf_matrix[1][0])}')
print(f'False Positive Rate: {conf_matrix[0][1]/(conf_matrix[0][0]+conf_matrix[0][1])}')

Confusion Matrix:
[[0.33905  0.021275]
 [0.433675 0.206   ]]
True Positive Rate: 0.322038535193653
False Positive Rate: 0.05904391868452091


In [12]:
# KNN
knn_wp = KNeighborsClassifier(n_neighbors=99)
knn_wp.fit(f_train, wp_r_train)

r_pred = knn_wp.predict(f_test)

conf_matrix = confusion_matrix(wp_r_test, r_pred)/f_test.shape[0]

print(f'Confusion Matrix:\n{conf_matrix}')
print(f'True Positive Rate: {conf_matrix[1][1]/(conf_matrix[1][1]+conf_matrix[1][0])}')
print(f'False Positive Rate: {conf_matrix[0][1]/(conf_matrix[0][0]+conf_matrix[0][1])}')

# adjust threshold
probabilities = knn_wp.predict_proba(f_test)[:, 1]

fpr, tpr, threshold = roc_curve(wp_r_test, probabilities)

selected_threshold = threshold[np.argmin(np.abs(fpr - 0.05))]

r_pred_adjusted = (probabilities >= selected_threshold).astype(int)

conf_matrix_adjusted = confusion_matrix(wp_r_test, r_pred_adjusted)/f_test.shape[0]

print(f'Adjusted Confusion Matrix:\n{conf_matrix_adjusted}')
print(f'True Positive Rate: {conf_matrix_adjusted[1][1]/(conf_matrix_adjusted[1][1]+conf_matrix_adjusted[1][0])}')
print(f'False Positive Rate: {conf_matrix_adjusted[0][1]/(conf_matrix_adjusted[0][0]+conf_matrix_adjusted[0][1])}')

Confusion Matrix:
[[0.218975 0.14135 ]
 [0.140025 0.49965 ]]
True Positive Rate: 0.781099777230625
False Positive Rate: 0.39228474294040105
Adjusted Confusion Matrix:
[[0.341525 0.0188  ]
 [0.412675 0.227   ]]
True Positive Rate: 0.3548677062570837
False Positive Rate: 0.05217511968361896


In [13]:
# Random Forest
rf_wp = RandomForestClassifier(n_estimators=100, random_state=73)
rf_wp.fit(f_train, wp_r_train)

r_pred = rf_wp.predict(f_test)

conf_matrix = confusion_matrix(wp_r_test, r_pred)/f_test.shape[0]

print(f'Confusion Matrix:\n{conf_matrix}')
print(f'True Positive Rate: {conf_matrix[1][1]/(conf_matrix[1][1]+conf_matrix[1][0])}')
print(f'False Positive Rate: {conf_matrix[0][1]/(conf_matrix[0][0]+conf_matrix[0][1])}')

# adjust threshold
probabilities = rf_wp.predict_proba(f_test)[:, 1]

fpr, tpr, threshold = roc_curve(wp_r_test, probabilities)

selected_threshold_wp = threshold[np.argmin(np.abs(fpr - 0.05))]

r_pred_adjusted = (probabilities >= selected_threshold_wp).astype(int)

conf_matrix_adjusted = confusion_matrix(wp_r_test, r_pred_adjusted)/f_test.shape[0]

print(f'Adjusted Confusion Matrix:\n{conf_matrix_adjusted}')
print(f'True Positive Rate: {conf_matrix_adjusted[1][1]/(conf_matrix_adjusted[1][1]+conf_matrix_adjusted[1][0])}')
print(f'False Positive Rate: {conf_matrix_adjusted[0][1]/(conf_matrix_adjusted[0][0]+conf_matrix_adjusted[0][1])}')

Confusion Matrix:
[[0.2249   0.135425]
 [0.092175 0.5475  ]]
True Positive Rate: 0.8559033884394419
False Positive Rate: 0.37584125442309024
Adjusted Confusion Matrix:
[[0.34245  0.017875]
 [0.322675 0.317   ]]
True Positive Rate: 0.49556415367178647
False Positive Rate: 0.04960799278429196


In [14]:
# isolate 'edited_2023' response variable
e2_r_train = r_train['edited_2023']
e2_r_test = r_test['edited_2023']

In [15]:
# Decision Tree
dtc_e2 = DecisionTreeClassifier()
dtc_e2.fit(f_train, e2_r_train)

r_pred = dtc_e2.predict(f_test)

conf_matrix = confusion_matrix(e2_r_test, r_pred)/f_test.shape[0]

print(f'Confusion Matrix:\n{conf_matrix}')

print(f'True Positive Rate: {conf_matrix[1][1]/(conf_matrix[1][1]+conf_matrix[1][0])}')

print(f'False Positive Rate: {conf_matrix[0][1]/(conf_matrix[0][0]+conf_matrix[0][1])}')

print()

# adjust threshold
probabilities = dtc_e2.predict_proba(f_test)[:, 1]

fpr, tpr, threshold = roc_curve(wp_r_test, probabilities)

selected_threshold = threshold[np.argmin(np.abs(fpr - 0.05))]

r_pred_adjusted = (probabilities >= selected_threshold).astype(int)

conf_matrix_adjusted = confusion_matrix(e2_r_test, r_pred_adjusted)/f_test.shape[0]

print(f'Adjusted Confusion Matrix:\n{conf_matrix_adjusted}')
print(f'True Positive Rate: {conf_matrix_adjusted[1][1]/(conf_matrix_adjusted[1][1]+conf_matrix_adjusted[1][0])}')
print(f'False Positive Rate: {conf_matrix_adjusted[0][1]/(conf_matrix_adjusted[0][0]+conf_matrix_adjusted[0][1])}')

Confusion Matrix:
[[0.17755  0.128675]
 [0.183825 0.50995 ]]
True Positive Rate: 0.7350365752585493
False Positive Rate: 0.42019756714833867

Adjusted Confusion Matrix:
[[0.306225 0.      ]
 [0.693775 0.      ]]
True Positive Rate: 0.0
False Positive Rate: 0.0


In [16]:
# Logistic Regression
model_logr_e2 = LogisticRegression(max_iter=1000, class_weight='balanced')
model_logr_e2.fit(f_train, e2_r_train)

r_pred = model_logr_e2.predict(f_test)

conf_matrix = confusion_matrix(e2_r_test, r_pred)/f_test.shape[0]

print(f'Confusion Matrix:\n{conf_matrix}')

print(f'True Positive Rate: {conf_matrix[1][1]/(conf_matrix[1][1]+conf_matrix[1][0])}')

print(f'False Positive Rate: {conf_matrix[0][1]/(conf_matrix[0][0]+conf_matrix[0][1])}')

print()

# adjust threshold
probabilities = model_logr_e2.predict_proba(f_test)[:, 1]

fpr, tpr, threshold = roc_curve(wp_r_test, probabilities)

selected_threshold = threshold[np.argmin(np.abs(fpr - 0.05))]

r_pred_adjusted = (probabilities >= selected_threshold).astype(int)

conf_matrix_adjusted = confusion_matrix(e2_r_test, r_pred_adjusted)/f_test.shape[0]

print(f'Adjusted Confusion Matrix:\n{conf_matrix_adjusted}')
print(f'True Positive Rate: {conf_matrix_adjusted[1][1]/(conf_matrix_adjusted[1][1]+conf_matrix_adjusted[1][0])}')
print(f'False Positive Rate: {conf_matrix_adjusted[0][1]/(conf_matrix_adjusted[0][0]+conf_matrix_adjusted[0][1])}')

Confusion Matrix:
[[0.24515  0.061075]
 [0.3223   0.371475]]
True Positive Rate: 0.5354401643184029
False Positive Rate: 0.1994448526410319

Adjusted Confusion Matrix:
[[0.29515  0.011075]
 [0.538025 0.15575 ]]
True Positive Rate: 0.22449641454362004
False Positive Rate: 0.03616621765042044


In [17]:
# Support Vector Machine
svm_e2 = LinearSVC(max_iter=100000, dual=False, class_weight={0:1, 1:0.25})
svm_e2.fit(f_train, e2_r_train)

r_pred = svm_e2.predict(f_test)

conf_matrix = confusion_matrix(e2_r_test, r_pred)/f_test.shape[0]

print(f'Confusion Matrix:\n{conf_matrix}')
print(f'True Positive Rate: {conf_matrix[1][1]/(conf_matrix[1][1]+conf_matrix[1][0])}')
print(f'False Positive Rate: {conf_matrix[0][1]/(conf_matrix[0][0]+conf_matrix[0][1])}')

Confusion Matrix:
[[0.28925  0.016975]
 [0.501525 0.19225 ]]
True Positive Rate: 0.27710713127454867
False Positive Rate: 0.05543309657931259


In [18]:
# KNN
knn_e2 = KNeighborsClassifier(n_neighbors=99)
knn_e2.fit(f_train, e2_r_train)

r_pred = knn_e2.predict(f_test)

conf_matrix = confusion_matrix(e2_r_test, r_pred)/f_test.shape[0]

print(f'Confusion Matrix:\n{conf_matrix}')
print(f'True Positive Rate: {conf_matrix[1][1]/(conf_matrix[1][1]+conf_matrix[1][0])}')
print(f'False Positive Rate: {conf_matrix[0][1]/(conf_matrix[0][0]+conf_matrix[0][1])}')

# adjust threshold
probabilities = knn_e2.predict_proba(f_test)[:, 1]

fpr, tpr, threshold = roc_curve(e2_r_test, probabilities)

selected_threshold = threshold[np.argmin(np.abs(fpr - 0.05))]

r_pred_adjusted = (probabilities >= selected_threshold).astype(int)

conf_matrix_adjusted = confusion_matrix(e2_r_test, r_pred_adjusted)/f_test.shape[0]

print(f'Adjusted Confusion Matrix:\n{conf_matrix_adjusted}')
print(f'True Positive Rate: {conf_matrix_adjusted[1][1]/(conf_matrix_adjusted[1][1]+conf_matrix_adjusted[1][0])}')
print(f'False Positive Rate: {conf_matrix_adjusted[0][1]/(conf_matrix_adjusted[0][0]+conf_matrix_adjusted[0][1])}')

Confusion Matrix:
[[0.12495  0.181275]
 [0.1074   0.586375]]
True Positive Rate: 0.8451947677561168
False Positive Rate: 0.5919666911584619
Adjusted Confusion Matrix:
[[0.291625 0.0146  ]
 [0.516675 0.1771  ]]
True Positive Rate: 0.25527008035746457
False Positive Rate: 0.04767736141725855


In [19]:
# Random Forest
rf_e2 = RandomForestClassifier(n_estimators=100, random_state=73)
rf_e2.fit(f_train, e2_r_train)

r_pred = rf_e2.predict(f_test)

conf_matrix = confusion_matrix(e2_r_test, r_pred)/f_test.shape[0]

print(f'Confusion Matrix:\n{conf_matrix}')
print(f'True Positive Rate: {conf_matrix[1][1]/(conf_matrix[1][1]+conf_matrix[1][0])}')
print(f'False Positive Rate: {conf_matrix[0][1]/(conf_matrix[0][0]+conf_matrix[0][1])}')

# adjust threshold
probabilities = rf_e2.predict_proba(f_test)[:, 1]

fpr, tpr, threshold = roc_curve(e2_r_test, probabilities)

selected_threshold_e2 = threshold[np.argmin(np.abs(fpr - 0.05))]

r_pred_adjusted = (probabilities >= selected_threshold_e2).astype(int)

conf_matrix_adjusted = confusion_matrix(e2_r_test, r_pred_adjusted)/f_test.shape[0]

print(f'Adjusted Confusion Matrix:\n{conf_matrix_adjusted}')
print(f'True Positive Rate: {conf_matrix_adjusted[1][1]/(conf_matrix_adjusted[1][1]+conf_matrix_adjusted[1][0])}')
print(f'False Positive Rate: {conf_matrix_adjusted[0][1]/(conf_matrix_adjusted[0][0]+conf_matrix_adjusted[0][1])}')

Confusion Matrix:
[[0.1477   0.158525]
 [0.093475 0.6003  ]]
True Positive Rate: 0.865266116536341
False Positive Rate: 0.5176749122377338
Adjusted Confusion Matrix:
[[0.290925 0.0153  ]
 [0.4317   0.262075]]
True Positive Rate: 0.3777521530755648
False Positive Rate: 0.04996326230712712


In [20]:
df_test = pd.read_pickle('./test_data.pkl')
# dataset sanity check
print(f'Shape: {df_test.shape}')
nan_counts = df_test.isna().sum()
print(nan_counts)
df_test.head()

Shape: (50000, 41)
URLID             0
educat         1018
polit           941
people          995
anger          1050
human           991
excess          973
city            988
ocean           920
math            968
social          988
rest            985
gram            976
analysis        985
data           1025
process         984
problem        1037
example        1002
sales           999
product        1015
question       1010
qualit          980
categor         945
first          1000
transaction    1048
survey          990
optim          1017
number          980
conduct        1045
experienc       987
visual          995
audio           947
elect          1047
decision        973
petrol         1041
mouse          1009
danger         1044
heart           960
soul           1002
grade           966
elderly        1000
dtype: int64


,URLID,educat,polit,people,anger,human,excess,city,ocean,math,...,audio,elect,decision,petrol,mouse,danger,heart,soul,grade,elderly
61630,URLID_GNPF,1.0,1.0,NaN,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32747,URLID_IIAA,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
231181,URLID_OOXK,0.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
152952,URLID_HCWK,0.0,0.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
22218,URLID_RXZL,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
selected_columns = df_test.columns[1:]
medians = df_test[selected_columns].median()
df_test[selected_columns] = df_test[selected_columns].fillna(medians)
filled_nan_counts = df_test.isna().sum()
print(filled_nan_counts)
df_test.head()

URLID          0
educat         0
polit          0
people         0
anger          0
human          0
excess         0
city           0
ocean          0
math           0
social         0
rest           0
gram           0
analysis       0
data           0
process        0
problem        0
example        0
sales          0
product        0
question       0
qualit         0
categor        0
first          0
transaction    0
survey         0
optim          0
number         0
conduct        0
experienc      0
visual         0
audio          0
elect          0
decision       0
petrol         0
mouse          0
danger         0
heart          0
soul           0
grade          0
elderly        0
dtype: int64


,URLID,educat,polit,people,anger,human,excess,city,ocean,math,...,audio,elect,decision,petrol,mouse,danger,heart,soul,grade,elderly
61630,URLID_GNPF,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32747,URLID_IIAA,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
231181,URLID_OOXK,0.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
152952,URLID_HCWK,0.0,0.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
22218,URLID_RXZL,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
def get_predictions(model, df, th):
    probabilities = model.predict_proba(df)[:, 1]
    
    return (probabilities >= th).astype(int)

In [23]:
output_file = 'final_part2.csv'
# isolate features
features = df_test.drop('URLID', axis=1).values
scaler_test = StandardScaler()
features = scaler_test.fit_transform(features)

# make predictions on test dataset
l_pred = model_dtr.predict(features)
wp_pred = get_predictions(rf_wp, features, selected_threshold_wp)
e2_pred = get_predictions(rf_e2, features, selected_threshold_e2)

output_df = df_test[['URLID']].copy()
output_df['length'] = l_pred
output_df['word_present'] = wp_pred
output_df['edited_2023'] = e2_pred

output_df.to_csv(output_file, index=False)

output_df.head()

,URLID,length,word_present,edited_2023
61630,URLID_GNPF,19961.0,1,1
32747,URLID_IIAA,33490.0,1,0
231181,URLID_OOXK,32132.0,1,1
152952,URLID_HCWK,43180.0,0,0
22218,URLID_RXZL,35918.0,0,1
